In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col, lit, regexp_replace


# New API
spark_session = SparkSession\
        .builder\
        .master("spark://130.238.28.143:7077") \
        .appName("Andreas Kladd")\
        .config("spark.dynamicAllocation.enabled",False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","99999s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

#        .config("spark.dynamicAllocation.shuffleTracking.enabled",False)\
#        .config("spark.shuffle.service.enabled",False)\

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/21 14:42:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#TODO

#importera timing library

#profanity per subreddit %
    #which subreddit curses the worst?
    #most common curse over-all?
    
#skriv timings till fil
    

In [5]:
df_dirty = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.13:9000/user/ubuntu/DirtyWordsFolder/DirtyWords.csv')

In [6]:
list_dirty = df_dirty.select("word").collect()
list_dirty = [str(row["word"]) for row in list_dirty]
print(list_dirty[1:5])

['anus', 'arse', 'ass', 'balls']


In [3]:
df_reddit = spark_session.read.json('hdfs://192.168.2.13:9000/user/one_worker')
df_reddit = df_reddit.select("body","subreddit")

In [4]:
rdd_reddit = df_reddit.select(concat(col("subreddit"),lit(","), col("body"))\
                                    .alias("SubredditAndBody"))\
                                    .rdd
rdd_reddit.take(2)

[Row(SubredditAndBody='IAmA,[deleted]'),
 Row(SubredditAndBody='programming,Thanks Oracle salesman!\n\nOh, wait.')]

In [ ]:
df_clean_text=rdd_reddit\
            .toDF()\
            .withColumn("Edited",regexp_replace("SubredditAndBody", ",|\n|\!|\?|\.|\*|:|;", " "))\
            .select("Edited")

rdd_split_text = df_clean_text.rdd.map(lambda row: row["Edited"].lower().split(' '))
                           

#result = " ".join(clean_text.split(' '))
df_clean_text.take(2)

[Row(Edited='IAmA [deleted]'),
 Row(Edited='programming Thanks Oracle salesman   Oh  wait ')]

In [21]:
def checkForDirtyWord(row, list_dirty):
    for word in row[1:]:
        if word in list_dirty:
            return True
    return False
    
df_dirty = rdd_split_text.filter(lambda row: checkForDirtyWord(row, list_dirty))
df_dirty.take(1)

subreddits = rdd.map(lambda row: row[0])

[['marijuana',
  "there's",
  'plenty',
  'of',
  'ass',
  'between',
  'the',
  'hole',
  'and',
  'your',
  'boxers',
  '',
  'not',
  'being',
  'mean',
  '',
  'but',
  'just',
  "sayin'",
  '',
  '',
  '',
  '',
  '']]

In [24]:
def counter(rdd):
    # take the first word = keyword
    assign = subreddits.map(lambda word: (word,1))
    add = assign.reduceByKey(lambda x, y: x+y)
    sort = add.map(lambda x: (x[1], x[0])).sortByKey(False)
    
    return sort

r = counter(df_dirty)
r.take(3)

[(56222, 'askreddit'), (22920, 'reddit'), (22290, 'iama')]

In [45]:
notreddit = df_reddit.rdd.filter(lambda row: row["subreddit"] != "reddit.com" and '.' in row["subreddit"])
notreddit.take(5)

[]

In [66]:
df_edited_reddit=df_reddit\
            .withColumn("subreddit",regexp_replace("subreddit", "reddit.com","reddit"))

df_edited_reddit.registerTempTable("df_edited_reddit")
subreddit_dirty_freq = spark_session.sql(
    "SELECT subreddit, COUNT(*) as frequencies FROM df_edited_reddit GROUP BY subreddit ORDER BY Frequencies DESC"
)

In [65]:
subreddit_freq = r.toDF(["frequencies", "subreddit"])

In [74]:
subreddit_freq.registerTempTable("subreddit_freq")
subreddit_dirty_freq.registerTempTable("subreddit_dirty_freq")

joined = spark_session.sql(
    "SELECT ISNULL(frequencies_sdf, 0) FROM (SELECT sf.subreddit, sf.frequencies, sdf.frequencies AS frequncies_sdf FROM subreddit_freq AS sf LEFT JOIN subreddit_dirty_freq AS sdf ON sf.subreddit = sdf.subreddit)" 
).show()




AnalysisException: cannot resolve 'frequencies_sdf' given input columns: [__auto_generated_subquery_name.frequencies, __auto_generated_subquery_name.frequncies_sdf, __auto_generated_subquery_name.subreddit]; line 1 pos 14;
'Project [unresolvedalias('ISNULL('frequencies_sdf, 0), None)]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [subreddit#332, frequencies#331L, frequencies#338L AS frequncies_sdf#403L]
      +- Join LeftOuter, (subreddit#332 = subreddit#335)
         :- SubqueryAlias sf
         :  +- SubqueryAlias subreddit_freq
         :     +- View (`subreddit_freq`, [frequencies#331L,subreddit#332])
         :        +- LogicalRDD [frequencies#331L, subreddit#332], false
         +- SubqueryAlias sdf
            +- SubqueryAlias subreddit_dirty_freq
               +- View (`subreddit_dirty_freq`, [subreddit#335,frequencies#338L])
                  +- Sort [Frequencies#338L DESC NULLS LAST], true
                     +- Aggregate [subreddit#335], [subreddit#335, co

In [70]:
print(type(joined))

<class 'pyspark.sql.dataframe.DataFrame'>


In [56]:
# release the cores for another application!
spark_context.stop()